# Join через merge колличество каждого MCC кода по client_id

In [34]:
pd.merge(
    transactions_train.reset_index(),
    transactions_train.groupby(['client_id', 'mcc_code'])['mcc_code'].count().rename('mcc_code_count').reset_index(),
    on=['client_id', 'mcc_code'],
    how="inner"
).set_index('client_id')

,trans_time,mcc_code,trans_type,amount,term_id,trans_city,gender,weekday,amount_up,amount_down,mcc_code_count
client_id,,,,,,,,,,,
0002cf30347684df542e1a931f356875,2020-06-10 14:52:03,4829,2370,-2170.07,888990,Saint Petersburg,0,2,NaN,2170.07,107
0002cf30347684df542e1a931f356875,2020-10-15 22:02:27,4829,2330,-2893.20,800926,Saint Petersburg,0,3,NaN,2893.20,107
0002cf30347684df542e1a931f356875,2020-07-30 07:44:21,4829,2331,-5011.43,10298153,Saint Petersburg,0,3,NaN,5011.43,107
0002cf30347684df542e1a931f356875,2020-08-15 11:37:57,4829,2331,-20761.51,980004,Saint Petersburg,0,5,NaN,20761.51,107
0002cf30347684df542e1a931f356875,2020-08-12 18:16:56,4829,2330,-26032.85,800925,Saint Petersburg,0,2,NaN,26032.85,107
...,...,...,...,...,...,...,...,...,...,...,...
fffedf876a0ea3d39e54b706165a4826,2020-02-06 15:05:52,6011,2010,-2169.75,NaN,Saint Petersburg,1,3,NaN,2169.75,61
fffedf876a0ea3d39e54b706165a4826,2019-08-12 12:35:42,6011,2010,-723.14,NaN,Saint Petersburg,1,0,NaN,723.14,61
fffedf876a0ea3d39e54b706165a4826,2019-08-18 09:55:37,5411,1010,-123.92,NaN,Saint Petersburg,1,6,NaN,123.92,1


# Разбивка на train и test

In [93]:
razbivka = transactions_train.groupby(['gender', 'client_id']).count().reset_index().set_index('client_id').sample(n=1250)
train = transactions_train[~transactions_train.index.isin(razbivka.index)]
test = transactions_train[transactions_train.index.isin(razbivka.index)]

In [94]:
display(train, test)

,trans_time,mcc_code,trans_type,amount,term_id,trans_city,gender,weekday,amount_up,amount_down
client_id,,,,,,,,,,
0002cf30347684df542e1a931f356875,2020-06-10 14:52:03,4829,2370,-2170.07,888990,Saint Petersburg,0,2,NaN,2170.07
0002cf30347684df542e1a931f356875,2019-12-30 14:10:49,6011,2010,-1445.64,NaN,Saint Petersburg,0,0,NaN,1445.64
0002cf30347684df542e1a931f356875,2019-12-02 12:38:32,5912,1010,-107.07,NaN,Saint Petersburg,0,0,NaN,107.07
0002cf30347684df542e1a931f356875,2020-01-08 13:22:34,6011,2010,-2892.86,NaN,Saint Petersburg,0,2,NaN,2892.86
0002cf30347684df542e1a931f356875,2020-04-15 12:06:54,5912,1010,-164.49,469965,Saint Petersburg,0,2,NaN,164.49
...,...,...,...,...,...,...,...,...,...,...
fffedf876a0ea3d39e54b706165a4826,2019-08-30 12:51:47,6011,2010,-10846.43,NaN,Saint Petersburg,1,4,NaN,10846.43
fffedf876a0ea3d39e54b706165a4826,2019-09-26 07:37:08,6011,2010,-2170.15,NaN,Saint Petersburg,1,3,NaN,2170.15
fffedf876a0ea3d39e54b706165a4826,2019-09-26 07:40:10,4814,1030,-71.41,NaN,Saint Petersburg,1,3,NaN,71.41


,trans_time,mcc_code,trans_type,amount,term_id,trans_city,gender,weekday,amount_up,amount_down
client_id,,,,,,,,,,
0002f4661e0d4d2ae097d78552dad022,2019-11-11 17:15:59,5411,1110,-75.04,NaN,Novosibirsk,0,0,NaN,75.04
0002f4661e0d4d2ae097d78552dad022,2020-06-05 20:57:38,5499,1010,-56.16,543894,Novosibirsk,0,4,NaN,56.16
0002f4661e0d4d2ae097d78552dad022,2020-06-19 14:02:54,6011,2010,-145.32,322185,Novosibirsk,0,4,NaN,145.32
0002f4661e0d4d2ae097d78552dad022,2020-07-24 17:38:19,4816,1100,-71.82,03170931,Novosibirsk,0,4,NaN,71.82
0002f4661e0d4d2ae097d78552dad022,2020-02-29 20:29:06,5813,1110,-143.63,569557,Novosibirsk,0,5,NaN,143.63
...,...,...,...,...,...,...,...,...,...,...
fffbed6c440a05e5593ad1a865e586a8,2020-01-23 15:17:00,5331,1010,-44.75,NaN,Vladivostok,1,3,NaN,44.75
fffbed6c440a05e5593ad1a865e586a8,2020-08-24 22:48:37,4814,1030,-44.26,888986,Vladivostok,1,0,NaN,44.26
fffbed6c440a05e5593ad1a865e586a8,2020-05-12 00:00:40,5541,1110,-650.67,00480618,Vladivostok,1,1,NaN,650.67
